In [ ]:
#Librerias:

from bs4 import BeautifulSoup
import requests
from deep_translator import GoogleTranslator
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os

In [ ]:
#Traer la página on bs4 para extraer los enlaces:

origin = 'https://www.classcentral.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
response = requests.get(origin, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

## Se buscan los enlaces
url_original = []
for link in soup.body.find_all("a"):
    enlace = link.get("href")
    if enlace is not None:
      url_original.append(enlace)
url_original.pop()

In [ ]:
#Se organizan los enlaces
i = 0
url_todos = []
for enlace in url_original:
  if enlace.startswith("/"):
    nueva_url = "https://www.classcentral.com" + enlace
    url_todos.append(nueva_url)
  i+=1

In [ ]:
#Se toman los enlaces unicos
url_unique = []
for i in url_todos:
  if i in url_unique:
    continue
  else:
    url_unique.append(i)

In [ ]:
#Se crea diccionario para facilitar la busqueda del enlace y a qué corresponde
enlaces = {}
for enlace in url_unique:
  valor = enlace
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces[dic] = enlace
  else:
    enlaces[dic] = enlace

In [ ]:
enlaces2 = {}
for enlace in url_unique:
  dic = enlace.replace("https://www.classcentral.com/", "")
  if dic == '':
    dic = "classcentral"
    enlaces2[dic.replace("/","_")] = enlace
  else:
    enlaces2[dic.replace("/","_")] = enlace

In [ ]:
dic3 = {}
for enlace in url_original:
  value = enlace
  if enlace.startswith("/"):
      enlace = enlace.replace("/","")
  dic = enlace.replace("https://www.classcentral.com/", "")
  
  if dic == '':
    dic = "https://cifu88.github.io/CodingAllStars-Test.github.io/classcentral.html"
    dic3[dic] = value
  else:
    dic = dic.replace("/","_") + ".html"
    key = "https://cifu88.github.io/CodingAllStars-Test.github.io/"+dic
    dic3[key] = value

    

In [ ]:
#Traducir y descargar todos los html:
driver = webdriver.Chrome()
try:      
  for i in enlaces2:
    print(i)
    filename = str(i) + ".html"
    if os.path.exists(filename):
    # Si el archivo ya existe, continuar con la siguiente iteración
      continue
    driver.get(enlaces2[i])
    time.sleep(2)

        ## Copiar el código fuente de la página
    look = driver.page_source
        
        #Transformar para capturar datos del html
    soup = BeautifulSoup(look, 'html.parser')
    html_tag = soup.html
    html_tag['lang'] = "hi"
           
    traductor = GoogleTranslator(source='auto', target='hi')
    trans = {}

        #Obtener texto traducido
    text = soup.stripped_strings

    for child in text:
      if child.isdigit() or child == 'Copy link' or len(child) <= 0:
        continue
      else:
        traduccion = (traductor.translate(child))
        trans[str(child)]=str(traduccion)
        
        #Valores unicos para traducir      
    texto = list(soup.stripped_strings)
    texto2 = []
    for palabra in texto:
      if palabra in texto2:
        continue
      else:
        texto2.append(palabra)
        #traduccion de la pagina
    for l in range (len(texto2)):
      for texto in soup.find_all(string=True):
        if texto.parent.name != 'style' and texto.parent.name != 'script':
          if texto2[l] in trans.keys():
            nuevo_texto = texto.replace(texto2[l], trans[texto2[l]])
            texto.replace_with(nuevo_texto)

        #Cambiar enlaces:
    links = soup.body.find_all("a")
    for s in dic3:
      for link in links:
        a = link.get('href')
        if a == dic3[s]:
          link['href'] =s
          

        #descargar archivo traducido:
    mensaje = str(soup)

    archivo = open(filename, "w", encoding="utf-8")
    archivo.write(mensaje)
    archivo.close()
except NameError:
    # Código que maneja la excepción NotValidPayload
    print("Error: NotValidPayload")

driver.close()